In [21]:
import json
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox
import random

%matplotlib inline

## Load and Clean Data

In [22]:
process_folder = '../../process'
process_config_path = '../../process/configuration/auckland.json'
pop_col = ["pop_ghs_2015"]
dest_col = ["destinations"]

In [23]:
with open(process_config_path) as json_file:
    config = json.load(json_file)

input_folder = os.path.join(process_folder, config['folder'])

# the path of "adelaide_au_2020_1600m_buffer.gpkg"
gpkg_input = os.path.join(input_folder, config['geopackagePath'])

In [24]:
pop = gpd.read_file(gpkg_input, layer='pop_ghs_2015' )

In [ ]:
pop.columns

In [ ]:
food_array = pop['count_fresh_food_market'].unique()

In [ ]:
food_array

In [25]:
dests = gpd.read_file(gpkg_input, layer='destinations' )

In [ ]:
dests.columns

In [ ]:
dests_array = dests['dest_name_full'].unique()

In [ ]:
dests_array

In [26]:
fresh_food = dests[dests['dest_name_full'].str.contains('Fresh Food / Market')]

In [27]:
gdf_study_area = gpd.read_file(gpkg_input, layer="urban_study_region")
study_area = gdf_study_area["geometry"].iloc[0]

In [28]:
crs = gdf_study_area.crs
if pop.crs != crs:
    pop = pop.to_crs(crs)
if fresh_food.crs != crs:
    fresh_food = fresh_food.to_crs(crs)

In [29]:
import warnings

warnings.filterwarnings("ignore", "GeoSeries.notna", UserWarning)  # temp warning suppression
pop_clipped = gpd.clip(pop, study_area)
fresh_food_clipped = gpd.clip(fresh_food, study_area)

## Create Density-Based Quintiles

In [30]:
pop_clipped['pop_quintile'] = pd.qcut(pop_clipped['pop_est'], 5, labels=False)

In [31]:
pop_clipped

,index,pop_est,area_sqkm,pop_per_sqkm,intersection_count,intersections_per_sqkm,count_pub,count_cafe,count_restaurant,count_convenience,count_bar,count_fast_food,count_food_court,count_fresh_food_market,count_pt_any,geometry,pop_quintile
0,2244,58.608079,0.040548,1445.400002,2,49.324258,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,"POLYGON ((1771577.000 5897635.000, 1771639.000...",1.0
1,8691,119.799445,0.040548,2954.509335,8,197.297031,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,"POLYGON ((1759301.000 5909189.000, 1759363.000...",1.0
2,3924,1.467867,0.040548,36.200734,2,49.324258,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1765439.000 5901886.000, 1765501.000...",0.0
3,7620,196.550784,0.040548,4847.360763,1,24.662129,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1767857.000 5907663.000, 1767919.000...",3.0
4,9720,0.000000,0.040548,0.000000,5,123.310644,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1766927.000 5910606.000, 1766989.000...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24648,25473,22.340064,0.040548,550.953527,7,172.634902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1755581.000 5938183.000, 1755629.545...",0.0
24649,25474,33.357764,0.040548,822.673473,10,246.621288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1755997.746 5938261.667, 1755953.000...",0.0
24650,25476,32.600246,0.040548,803.991462,1,24.662129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1756196.763 5938190.449, 1756157.906...",0.0
24651,25477,59.001475,0.040548,1455.101987,7,172.634902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1756564.778 5938197.455, 1756573.000...",1.0


In [32]:
quintile_0 = pop_clipped.loc[pop_clipped['pop_quintile'] == 0]
quintile_1 = pop_clipped.loc[pop_clipped['pop_quintile'] == 1]
quintile_2 = pop_clipped.loc[pop_clipped['pop_quintile'] == 2]
quintile_3 = pop_clipped.loc[pop_clipped['pop_quintile'] == 3]
quintile_4 = pop_clipped.loc[pop_clipped['pop_quintile'] == 4]

## Assign Hexagon-Data to Destinations

In [33]:
joined_freshfood = gpd.sjoin(fresh_food_clipped, pop_clipped, how='right', op='within')

In [ ]:
joined_freshfood

In [34]:
cleaned_joined_freshfood = joined_freshfood[~joined_freshfood['dest_name_full'].isnull()]

In [35]:
cleaned_joined_freshfood

,index_left,dest_oid,osm_id,dest_name,dest_name_full,edge_ogc_fid,n1,n2,n1_distance,n2_distance,...,count_cafe,count_restaurant,count_convenience,count_bar,count_fast_food,count_food_court,count_fresh_food_market,count_pt_any,geometry,pop_quintile
719,0.0,12.0,4989764212,fresh_food_market,Fresh Food / Market,98972.0,2.588550e+07,2.588549e+07,100.0,25.0,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2.0,"POLYGON ((1760231.000 5917800.000, 1760293.000...",3.0
17282,1.0,13.0,4753906102,fresh_food_market,Fresh Food / Market,45492.0,2.576970e+07,2.576970e+07,34.0,51.0,...,2.0,1.0,1.0,NaN,NaN,NaN,1.0,3.0,"POLYGON ((1756325.000 5919435.000, 1756387.000...",4.0
695,2.0,15.0,4662910730,fresh_food_market,Fresh Food / Market,41864.0,5.702913e+09,5.702913e+09,28.0,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,"POLYGON ((1765253.000 5916165.000, 1765315.000...",1.0
19379,3.0,20.0,5237396056,fresh_food_market,Fresh Food / Market,12251.0,5.721574e+09,5.721574e+09,14.0,31.0,...,NaN,1.0,1.0,NaN,1.0,NaN,1.0,2.0,"POLYGON ((1745909.000 5912677.000, 1745971.000...",4.0
19453,4.0,21.0,5237417285,fresh_food_market,Fresh Food / Market,89323.0,6.192534e+09,6.192534e+09,17.0,15.0,...,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,"POLYGON ((1746653.000 5912677.000, 1746715.000...",3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18568,7685.0,627.0,48985735,fresh_food_market,Fresh Food / Market,51369.0,4.120859e+08,3.230751e+09,69.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,"POLYGON ((1745351.000 5917582.000, 1745413.000...",3.0
603,7686.0,629.0,58841005,fresh_food_market,Fresh Food / Market,54359.0,7.051506e+09,7.051506e+09,11.0,28.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1745165.000 5919653.000, 1745227.000...",1.0
19418,7690.0,646.0,778743665,fresh_food_market,Fresh Food / Market,107820.0,2.526243e+09,2.526243e+09,33.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,"POLYGON ((1752658.774 5933917.539, 1752667.000...",0.0
10253,7693.0,667.0,329179983,fresh_food_market,Fresh Food / Market,99157.0,3.357253e+09,2.588590e+07,68.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,"POLYGON ((1762835.000 5914094.000, 1762897.000...",3.0


## Select Destinations

In [36]:
final_destiations = cleaned_joined_freshfood.sample(frac=1).groupby('pop_quintile', sort=False).head(10)

In [37]:
final_destiations

,index_left,dest_oid,osm_id,dest_name,dest_name_full,edge_ogc_fid,n1,n2,n1_distance,n2_distance,...,count_cafe,count_restaurant,count_convenience,count_bar,count_fast_food,count_food_court,count_fresh_food_market,count_pt_any,geometry,pop_quintile
17094,2645.0,212.0,3666557161,fresh_food_market,Fresh Food / Market,110048.0,5.683256e+09,5.683250e+09,17.0,45.0,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,"POLYGON ((1770275.000 5911914.000, 1770337.000...",2.0
17385,41.0,156.0,3696072720,fresh_food_market,Fresh Food / Market,96255.0,5.726725e+09,5.726725e+09,19.0,57.0,...,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,"POLYGON ((1764323.000 5909516.000, 1764385.000...",4.0
673,43.0,162.0,5029943787,fresh_food_market,Fresh Food / Market,96978.0,3.905058e+07,6.201616e+08,40.0,53.0,...,NaN,1.0,NaN,NaN,4.0,NaN,1.0,NaN,"POLYGON ((1763765.000 5910061.000, 1763827.000...",2.0
19446,80.0,251.0,4855260696,fresh_food_market,Fresh Food / Market,90137.0,5.550860e+09,6.224781e+09,16.0,21.0,...,NaN,NaN,2.0,NaN,2.0,NaN,2.0,2.0,"POLYGON ((1742933.000 5922269.000, 1742995.000...",1.0
721,172.0,521.0,5077967642,fresh_food_market,Fresh Food / Market,60583.0,3.010483e+08,3.010483e+08,137.0,48.0,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2.0,"POLYGON ((1766927.000 5918018.000, 1766989.000...",2.0
219,165.0,508.0,3663904497,fresh_food_market,Fresh Food / Market,97035.0,3.663916e+09,3.663916e+09,36.0,58.0,...,2.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,"POLYGON ((1769159.000 5916710.000, 1769221.000...",1.0
18176,7592.0,269.0,692964149,fresh_food_market,Fresh Food / Market,43654.0,7.787338e+08,4.192808e+09,36.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"POLYGON ((1747955.000 5913876.000, 1748017.000...",3.0
19450,2600.0,54.0,5304275724,fresh_food_market,Fresh Food / Market,106422.0,5.304439e+09,5.304439e+09,10.0,0.0,...,NaN,NaN,1.0,NaN,1.0,NaN,1.0,2.0,"POLYGON ((1768973.000 5901995.000, 1769035.000...",2.0
19449,5139.0,565.0,5057215420,fresh_food_market,Fresh Food / Market,90692.0,7.293806e+09,7.293806e+09,19.0,28.0,...,NaN,NaN,2.0,NaN,2.0,NaN,2.0,1.0,"POLYGON ((1756511.000 5932624.000, 1756573.000...",1.0
19465,116.0,343.0,3431091027,fresh_food_market,Fresh Food / Market,34342.0,1.319383e+08,2.383210e+09,26.0,89.0,...,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,"POLYGON ((1753349.000 5928809.000, 1753411.000...",4.0


## Checking Work

In [ ]:
cleaned_joined_freshfood['pop_quintile'].hist()
plt.show()

In [ ]:
scheme = 'Quantiles'
k = 5
cmap = 'plasma'
edgecolor = 'none'
city_color = 'none'
city_edge = 'w'
city_edge_lw = 0.2
title_y = 1.02
title_fontsize = 16
title_weight = 'bold'

fontcolor = 'w'
params = {"text.color" : fontcolor,
          "ytick.color" : fontcolor,
          "xtick.color" : fontcolor}
plt.rcParams.update(params)

In [ ]:
cleaned_joined_freshfood.plot(column='n1', cmap=cmap, edgecolor=edgecolor)

In [ ]:
pop_clipped.plot(column='pop_est', cmap=cmap, edgecolor=edgecolor)

In [ ]:
pop.plot(column='pop_est', cmap=cmap, edgecolor=edgecolor)

In [ ]:
pop.head(1).plot(column='pop_est', cmap=cmap, edgecolor=edgecolor)

In [ ]:
dests.plot(column='n1', cmap=cmap, edgecolor=edgecolor)

In [ ]:
grouped_quintiles = cleaned_joined_freshfood.groupby('pop_quintile')

In [ ]:
quintile_0adelaide_au_2020_1600m_buffer.gpkg